# Ejercicio 1

Implementar los siguientes métodos de descenso gradiente (naïve = tamaño de paso α constante):

- descenso gradiente naïve con dirección de descenso aleatoria  
- descenso máximo naïve  
- descenso grediente de Newton, con Hessiano exacto  
- un método de gradiente conjugado (Fletcher-Reeves, Hestenes-Stiefel, Polak-Ribière)  
- el método BFGS.  

En cada uno de los métodos, su función debe recibir los siguientes argumentos:  
 la función objetivo f,  
 el gradiente de la función objetivo df,  
 el hessiano ddf (cuando sea necesario),  
 un punto inicial x0 ∈ ℝⁿ,  
 el tamaño de paso α > 0,  
 el número máximo de iteraciones maxIter,  
 la tolerancia ε, así como un criterio de paro.  

Como resultado, sus algoritmos deben devolver:  
- la mejor solución encontrada `best` (la última de las aproximaciones calculadas),  
- la secuencia de iteraciones `xk`,  
- la secuencia de valores `f(xk)`,  
- la secuencia de errores en cada paso (según el error de su criterio de paro).  

Además, es deseable indicar:  
- el número de iteraciones efectuadas por el algoritmo,  
- si se obtuvo o no convergencia del método.  

In [1]:
# libs
import numpy as np

In [2]:
def descenso_gradiente_naive(f, df, x0, alpha=0.01, max_iter=1000, tol=1e-6):
    """
    Implementación simple del descenso gradiente con paso fijo (α constante).

    Args:
        f (función): Función objetivo a minimizar.
        df (función): Gradiente de la función.
        x0 (array): Punto inicial.
        alpha (float): Tamaño del paso (learning rate).
        max_iter (int): Máximo número de iteraciones.
        tol (float): Tolerancia para detenerse (si el cambio en f(x) es muy pequeño).

    Returns:
        mejor_x (array): Mejor solución encontrada.
        historial_x (list): Lista de todas las posiciones x visitadas.
        historial_f (list): Lista de valores f(x) en cada paso.
        convergio (bool): True si el método convergió antes de max_iter.
    """
    x_actual = x0.copy()
    historial_x = [x_actual.copy()]
    historial_f = [f(x_actual)]
    convergio = False

    for iteracion in range(max_iter):
        gradiente = df(x_actual)
        x_nuevo = x_actual - alpha * gradiente

        historial_x.append(x_nuevo.copy())
        historial_f.append(f(x_nuevo))

        if np.linalg.norm(x_nuevo - x_actual) < tol:
            convergio = True
            break

        x_actual = x_nuevo

    mejor_x = x_actual
    return mejor_x, historial_x, historial_f, convergio

In [3]:
def descenso_gradiente_aleatorio(f, x0, alpha=0.01, max_iter=1000, tol=1e-6):
    """
    Descenso gradiente con dirección aleatoria (no usa el gradiente real).
    """
    x_actual = x0.copy()
    historial_x = [x_actual.copy()]
    historial_f = [f(x_actual)]
    convergio = False

    for iteracion in range(max_iter):
        direccion_aleatoria = np.random.randn(*x0.shape)
        direccion_aleatoria = direccion_aleatoria / np.linalg.norm(direccion_aleatoria)

        x_nuevo = x_actual - alpha * direccion_aleatoria

        historial_x.append(x_nuevo.copy())
        historial_f.append(f(x_nuevo))

        if np.linalg.norm(x_nuevo - x_actual) < tol:
            convergio = True
            break

        x_actual = x_nuevo

    mejor_x = x_actual
    return mejor_x, historial_x, historial_f, convergio

In [4]:
def descenso_maximo_naive(f, df, x0, alpha=0.01, max_iter=1000, tol=1e-6):
    """
    Descenso máximo (usa el gradiente real, pero con paso fijo).
    """
    x_actual = x0.copy()
    historial_x = [x_actual.copy()]
    historial_f = [f(x_actual)]
    convergio = False

    for iteracion in range(max_iter):
        gradiente = df(x_actual)
        x_nuevo = x_actual - alpha * gradiente

        historial_x.append(x_nuevo.copy())
        historial_f.append(f(x_nuevo))

        if np.linalg.norm(gradiente) < tol:
            convergio = True
            break

        x_actual = x_nuevo

    mejor_x = x_actual
    return mejor_x, historial_x, historial_f, convergio

In [5]:
def newton(f, df, ddf, x0, max_iter=1000, tol=1e-6):
    """
    Método de Newton (usa el Hessiano para ajustar el paso).
    """
    x_actual = x0.copy()
    historial_x = [x_actual.copy()]
    historial_f = [f(x_actual)]
    convergio = False

    for iteracion in range(max_iter):
        gradiente = df(x_actual)
        hessiano = ddf(x_actual)

        paso = np.linalg.solve(hessiano, -gradiente)
        x_nuevo = x_actual + paso

        historial_x.append(x_nuevo.copy())
        historial_f.append(f(x_nuevo))

        if np.linalg.norm(gradiente) < tol:
            convergio = True
            break

        x_actual = x_nuevo

    mejor_x = x_actual
    return mejor_x, historial_x, historial_f, convergio

In [6]:
def gradiente_conjugado_fletcher_reeves(f, df, x0, max_iter=1000, tol=1e-6):
    """
    Gradiente conjugado (Fletcher-Reeves).
    """
    x_actual = x0.copy()
    historial_x = [x_actual.copy()]
    historial_f = [f(x_actual)]
    convergio = False

    gradiente_actual = df(x_actual)
    direccion = -gradiente_actual

    for iteracion in range(max_iter):
        alpha = 0.01
        x_nuevo = x_actual + alpha * direccion

        gradiente_nuevo = df(x_nuevo)

        beta = np.dot(gradiente_nuevo, gradiente_nuevo) / np.dot(gradiente_actual, gradiente_actual)
        direccion = -gradiente_nuevo + beta * direccion

        historial_x.append(x_nuevo.copy())
        historial_f.append(f(x_nuevo))

        if np.linalg.norm(gradiente_nuevo) < tol:
            convergio = True
            break

        x_actual = x_nuevo
        gradiente_actual = gradiente_nuevo

    mejor_x = x_actual
    return mejor_x, historial_x, historial_f, convergio

In [7]:
def bfgs(f, df, x0, max_iter=1000, tol=1e-6):
    """
    Método BFGS (aproxima el Hessiano usando diferencias de gradientes).
    """
    n = len(x0)
    H = np.eye(n)
    x_actual = x0.copy()
    historial_x = [x_actual.copy()]
    historial_f = [f(x_actual)]
    convergio = False

    gradiente_actual = df(x_actual)

    for iteracion in range(max_iter):
        paso = -H @ gradiente_actual

        alpha = 0.01
        x_nuevo = x_actual + alpha * paso

        gradiente_nuevo = df(x_nuevo)

        s = x_nuevo - x_actual
        y = gradiente_nuevo - gradiente_actual

        rho = 1.0 / (y.T @ s)
        H = (np.eye(n) - rho * np.outer(s, y)) @ H @ (np.eye(n) - rho * np.outer(y, s)) + rho * np.outer(s, s)

        historial_x.append(x_nuevo.copy())
        historial_f.append(f(x_nuevo))

        if np.linalg.norm(gradiente_nuevo) < tol:
            convergio = True
            break

        x_actual = x_nuevo
        gradiente_actual = gradiente_nuevo

    mejor_x = x_actual
    return mejor_x, historial_x, historial_f, convergio